In [1]:
# Download data
%run "../dataio/gtsrb_download.py"

2025-11-02 14:24:16,914 - INFO - Downloading GTSRB_Final_Training_Images...


KeyboardInterrupt: 

In [1]:
# Hard coded because I can't be bothered to make it dynamic LMAO
%cd c:\Users\Gebruiker\Documents\YEAR 3\Neural Networks\traffic-sign-classification

c:\Users\Gebruiker\Documents\YEAR 3\Neural Networks\traffic-sign-classification


In [2]:
from dataio.transforms import ToCompose, ToResize, ToRotate, ToNoise, ToTensor, ToNormalize
from dataio.gtsrb_dataset import GTSRBDataset
from dataio.dataloader import DataLoader
from nn.layers.batchnorm2d import BatchNorm2D
from nn.layers.conv2d import Conv2D
from nn.layers.dropout import Dropout
from nn.layers.flatten import Flatten
from nn.layers.linear import Linear
from nn.layers.maxpool2d import MaxPool2D
from nn.layers.sequential import Sequential
from nn.optim import Adam
from nn.loss import cross_entropy
from train import train
from crossval import cross_validate
import numpy as np

In [3]:
# Define the transforms for training
train_transforms = ToCompose([
    ToResize(size=64),
    ToRotate(angle=15),
    ToNoise(mean=0, std=0.05),
    ToTensor(),
    ToNormalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

# Define the transforms for validation and testing
val_transforms = ToCompose([
    ToResize(size=64),
    ToTensor(),
    ToNormalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

# %% [markdown]
# ## Load Data

# %%
# Total number of entries in the dataset
total_entries = 51840

# Define the indices for each split
def get_train_indices():
    return list(range(int(0.01 * total_entries)))

def get_val_indices():
    start = int(0.7 * total_entries)
    end = int(0.85 * total_entries)
    return list(range(start, end))

def get_test_indices():
    start = int(0.85 * total_entries)
    return list(range(start, total_entries))

# Initialize the dataset
train_dataset = GTSRBDataset(
    root="./data/gtsrb/",
    indices=get_train_indices(),
    split="train",
    transforms=train_transforms
)
val_dataset = GTSRBDataset(
    root="./data/gtsrb/",
    indices=get_val_indices(),
    split="val",
    transforms=val_transforms
)
test_dataset = GTSRBDataset(
    root="./data/gtsrb/",
    indices=get_test_indices(),
    split="test",
    transforms=val_transforms
)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [4]:
# Example architecture without ReLU
layers = [
    Conv2D(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1),
    BatchNorm2D(num_channels=32),
    MaxPool2D(pool_size=2, stride=2),
    Conv2D(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
    BatchNorm2D(num_channels=64),
    MaxPool2D(pool_size=2, stride=2),
    Conv2D(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
    BatchNorm2D(num_channels=128),
    MaxPool2D(pool_size=2, stride=2),
    Flatten(),
    Linear(in_features=128 * 8 * 8, out_features=512),  # Adjust input_size based on your image size and pooling layers
    Dropout(p=0.5),
    Linear(in_features=512, out_features=43)  # GTSRB has 43 classes
]

model = Sequential(layers)

In [ ]:
raw_params = {}
for i, layer in enumerate(model.layers):
    for name, param in layer.params().items():
        raw_params[f"layer{i}_{name}"] = param

optimizer = Adam(raw_params, lr=0.001)

# Define the loss function
loss_fn = cross_entropy

AttributeError: 'list' object has no attribute 'items'

In [ ]:

# Train the model using the train function
train_losses, val_losses, train_accs, val_accs = train(
    model, train_loader, val_loader, loss_fn, optimizer, num_epochs=10
)

2025-11-02 14:26:42,310 - DEBUG - Training started.
2025-11-02 14:26:42,310 - root - DEBUG - Training started.
2025-11-02 14:26:42,313 - DEBUG - Starting epoch 1/10.
2025-11-02 14:26:42,313 - root - DEBUG - Starting epoch 1/10.
2025-11-02 14:26:43,161 - DEBUG - Processing training batch with 128 samples.
2025-11-02 14:26:43,161 - root - DEBUG - Processing training batch with 128 samples.


KeyboardInterrupt: 

In [ ]:
results = cross_validate(
    model=model,
    optimizer=optimizer,
    loss_fn=loss_fn,
    checkpoint_root="./checkpoints/crossval/",
    log_dir="./logs/crossval/"
)

2025-11-02 14:26:57,736 - DEBUG - Starting fold 1/5.
2025-11-02 14:26:57,736 - root - DEBUG - Starting fold 1/5.
2025-11-02 14:26:57,736 - root - DEBUG - Starting fold 1/5.


2025-11-02 14:26:58,094 - DEBUG - Training started.
2025-11-02 14:26:58,094 - root - DEBUG - Training started.
2025-11-02 14:26:58,094 - root - DEBUG - Training started.
2025-11-02 14:26:58,094 - root - DEBUG - Training started.
2025-11-02 14:26:58,101 - DEBUG - Starting epoch 1/10.
2025-11-02 14:26:58,101 - root - DEBUG - Starting epoch 1/10.
2025-11-02 14:26:58,101 - root - DEBUG - Starting epoch 1/10.
2025-11-02 14:26:58,101 - root - DEBUG - Starting epoch 1/10.
2025-11-02 14:26:58,221 - DEBUG - Processing training batch with 32 samples.
2025-11-02 14:26:58,221 - root - DEBUG - Processing training batch with 32 samples.
2025-11-02 14:26:58,221 - root - DEBUG - Processing training batch with 32 samples.
2025-11-02 14:26:58,221 - root - DEBUG - Processing training batch with 32 samples.


41472 10368


2025-11-02 14:27:00,608 - DEBUG - Training batch loss: 7.702752763073777
2025-11-02 14:27:00,608 - root - DEBUG - Training batch loss: 7.702752763073777
2025-11-02 14:27:00,608 - root - DEBUG - Training batch loss: 7.702752763073777
2025-11-02 14:27:00,608 - root - DEBUG - Training batch loss: 7.702752763073777
2025-11-02 14:27:04,329 - DEBUG - Training batch accuracy: 0.03125
2025-11-02 14:27:04,329 - root - DEBUG - Training batch accuracy: 0.03125
2025-11-02 14:27:04,329 - root - DEBUG - Training batch accuracy: 0.03125
2025-11-02 14:27:04,329 - root - DEBUG - Training batch accuracy: 0.03125
2025-11-02 14:27:04,335 - DEBUG - Processing training batch with 32 samples.
2025-11-02 14:27:04,335 - root - DEBUG - Processing training batch with 32 samples.
2025-11-02 14:27:04,335 - root - DEBUG - Processing training batch with 32 samples.
2025-11-02 14:27:04,335 - root - DEBUG - Processing training batch with 32 samples.
2025-11-02 14:27:06,597 - DEBUG - Training batch loss: 16.07018275551

KeyboardInterrupt: 